In [3]:
import pandas as pd
import numpy as np
np.random.seed(0)
from nltk import word_tokenize
from gensim.models import word2vec

In [4]:
posts_df = pd.read_csv('data/posts.csv')
posts_df.head()

,Unnamed: 0,age,comment,likes/views,link,promo,user
0,0,2 DAYS AGO,found my new favorite park!,"405,059 likes",/BytNlrQhRx8/,0,chrissyteigen
1,1,3 DAYS AGO,Happy bebe!,"1,739,218",/Byqz8uZh73s/,0,chrissyteigen
2,2,5 DAYS AGO,coated in a paste of fresh garlic and filled w...,"2,931,603",/BymErW1B9eL/,0,chrissyteigen
3,3,5 DAYS AGO,this kid,"371,095 likes",/Byl-aHjBXFX/,0,chrissyteigen
4,4,JUNE 8,home tomorrow 😩,"859,039 likes",/ByduG0BB_A3/,0,chrissyteigen


In [5]:
posts_df.shape

(1065, 7)

In [6]:
target = posts_df.promo
data = posts_df['comment'].map(word_tokenize).values

In [7]:
total_vocabulary = set(word for comment in data for word in comment)
print("There are {} unique tokens in our dataset.".format(len(total_vocabulary)))

There are 8994 unique tokens in our dataset.


In [8]:
glove = {}
with open('data/glove_data/glove.6B.50d.txt', 'rb') as f:
    for line in f:
        parts = line.split()
        word = parts[0].decode('utf-8')
        if word in total_vocabulary:
            vector = np.array(parts[1:], dtype=np.float32)
            glove[word] = vector

In [9]:
glove['cool']

array([-0.65575 ,  0.45659 , -0.16748 , -0.58345 , -0.23073 , -0.78348 ,
       -0.23166 , -0.022452, -0.57968 ,  0.526   , -0.2214  ,  0.17614 ,
        0.46513 ,  0.79142 ,  0.017403,  1.0879  ,  0.24418 ,  0.27523 ,
       -0.26452 , -1.0389  ,  0.014045,  0.68459 ,  0.98151 ,  0.21561 ,
        0.36278 , -0.51819 , -0.40552 ,  1.349   ,  1.5399  ,  0.60541 ,
        2.6604  ,  0.074535, -0.076292,  0.12501 , -0.026268,  0.16843 ,
       -0.41844 ,  0.44505 ,  0.25033 , -1.1557  ,  0.24575 ,  0.41847 ,
       -0.10633 , -0.28433 ,  0.51215 ,  0.51371 ,  0.53004 , -0.889   ,
        0.054744,  0.78793 ], dtype=float32)

#### Create Mean Word Embeddings

In [10]:
class W2vVectorizer(object):
    
    def __init__(self, w2v):
        # takes in a dictionary of words and vectors as input
        self.w2v = w2v
        if len(w2v) == 0:
            self.dimensions = 0
        else:
            self.dimensions = len(w2v[next(iter(glove))])
    
    # Note from Mike: Even though it doesn't do anything, it's required that this object implement a fit method or else
    # It can't be used in a sklearn Pipeline. 
    def fit(self, X, y):
        return self
            
    def transform(self, X):
        return np.array([
            np.mean([self.w2v[w] for w in words if w in self.w2v]
                   or [np.zeros(self.dimensions)], axis=0) for words in X])

#### Using Pipelines

In [11]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_val_score

rf =  Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ("Random Forest", RandomForestClassifier(n_estimators=100, verbose=True))])
svc = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
                ('Support Vector Machine', SVC())])
lr = Pipeline([("Word2Vec Vectorizer", W2vVectorizer(glove)),
              ('Logistic Regression', LogisticRegression())])

In [12]:
models = [('Random Forest', rf),
          ("Support Vector Machine", svc),
          ("Logistic Regression", lr)]

In [13]:
scores = [(name, cross_val_score(model, data, target, cv=2).mean()) for name, model, in models]

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.2s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 100 out of 100 | elapsed:    0.0s finished
/Users/sherzyang/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:193: FutureWarning: The default value of gamma will change from 'auto' to 'scale' in version 0.22 to account better for unscaled features. Set gamma explicitly to 'auto' or 'scale' to avoid this warning.
  "avoid this warning.", FutureWarning)
/Users/sherzyang/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:19

In [14]:
scores

[('Random Forest', 0.6281722128962179),
 ('Support Vector Machine', 0.5624321121753728),
 ('Logistic Regression', 0.5877851288634344)]

#### Deep Learning with word embeddings

In [15]:
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.layers import Input, Dense, LSTM, Embedding
from tensorflow.keras.layers import Dropout, Activation, Bidirectional, GlobalMaxPool1D
from tensorflow.keras.models import Model
from tensorflow.keras import initializers, regularizers, constraints, optimizers, layers
from tensorflow.keras.preprocessing import text, sequence

In [16]:
y = pd.get_dummies(target).values

In [17]:
tokenizer = text.Tokenizer(num_words=20000)
tokenizer.fit_on_texts(list(posts_df.comment))
list_tokenized_headlines = tokenizer.texts_to_sequences(posts_df.comment)
X_t = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [18]:
from tensorflow.python.framework import ops
ops.reset_default_graph()

In [19]:
embedding_size = 128
input_ = Input(shape=(100,))
x = Embedding(20000, embedding_size)(input_)
x = Bidirectional(LSTM(25, return_sequences=True))(x)
x = GlobalMaxPool1D()(x)
x = Dense(50, activation='relu')(x)
x = Dropout(0.5)(x)
# There are 2 different possible classes, so we use 2 neurons in our output layer
x = Dense(2, activation='softmax')(x)

model = Model(inputs=input_, outputs=x)

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [20]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [21]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         (None, 100)               0         
_________________________________________________________________
embedding (Embedding)        (None, 100, 128)          2560000   
_________________________________________________________________
bidirectional (Bidirectional (None, 100, 50)           30800     
_________________________________________________________________
global_max_pooling1d (Global (None, 50)                0         
_________________________________________________________________
dense (Dense)                (None, 50)                2550      
_________________________________________________________________
dropout (Dropout)            (None, 50)                0         
_________________________________________________________________
dense_1 (Dense)              (None, 2)                 102       
Total para

In [22]:
model.fit(X_t, y, epochs=2, batch_size=32, validation_split=0.1)

Train on 958 samples, validate on 107 samples
Instructions for updating:
Use tf.cast instead.
Instructions for updating:
Deprecated in favor of operator or tf.math.divide.
Epoch 1/2
958/958 [==============================] - 5s 5ms/sample - loss: 0.6783 - acc: 0.5731 - val_loss: 0.6539 - val_acc: 0.6075
Epoch 2/2
958/958 [==============================] - 4s 4ms/sample - loss: 0.6431 - acc: 0.6242 - val_loss: 0.6179 - val_acc: 0.6636


In [23]:
model.fit(X_t, y, epochs=10, batch_size=32, validation_split=0.1)

Train on 958 samples, validate on 107 samples
Epoch 1/10
958/958 [==============================] - 4s 4ms/sample - loss: 0.4844 - acc: 0.8278 - val_loss: 0.5509 - val_acc: 0.7290
Epoch 2/10
958/958 [==============================] - 3s 4ms/sample - loss: 0.2322 - acc: 0.9363 - val_loss: 0.7209 - val_acc: 0.7196
Epoch 3/10
958/958 [==============================] - 4s 4ms/sample - loss: 0.1483 - acc: 0.9468 - val_loss: 0.8232 - val_acc: 0.5514
Epoch 4/10
958/958 [==============================] - 4s 4ms/sample - loss: 0.0867 - acc: 0.9635 - val_loss: 0.8118 - val_acc: 0.6916
Epoch 5/10
958/958 [==============================] - 4s 4ms/sample - loss: 0.0268 - acc: 0.9979 - val_loss: 0.9954 - val_acc: 0.7009
Epoch 6/10
958/958 [==============================] - 4s 4ms/sample - loss: 0.0244 - acc: 0.9916 - val_loss: 1.2244 - val_acc: 0.7103
Epoch 7/10
958/958 [==============================] - 4s 5ms/sample - loss: 0.0111 - acc: 0.9969 - val_loss: 1.3520 - val_acc: 0.6636
Epoch 8/10
958/9

In [24]:
import pickle 
test_df = pickle.load(open('data/testing_posts.pkl', 'rb'))

In [25]:
list_tokenized_headlines = tokenizer.texts_to_sequences(test_df['comment'].map(word_tokenize).values)
testing_X = sequence.pad_sequences(list_tokenized_headlines, maxlen=100)

In [26]:
testing_y = pd.get_dummies(test_df['promo']).values

In [27]:
model.evaluate(testing_X, testing_y, verbose=True)

248/248 [==============================] - 0s 763us/sample - loss: 1.7697 - acc: 0.6774


[1.7696544431870984, 0.67741936]

In [40]:
predictions = model.predict(testing_X).round(2)

In [48]:
predictions_df = pd.DataFrame(predictions, columns=['pred_not_promo', 'pred_promo'])
simple_test = test_df[['comment', 'promo']]
combined_posts_df = pd.concat((simple_test, predictions_df), axis=1)
combined_posts_df.head()

,comment,promo,pred_not_promo,pred_promo
0,Summer is a great time to freshen up the kids’...,1,0.01,0.99
1,They all look up to you so much @chipgaines Yo...,0,1.00,0.00
2,Happy Father's Day to the plant daddy of all p...,0,1.00,0.00
3,Y’all really stepped up to the #ChipInChalleng...,0,0.26,0.74
4,One more push y’all! We have til midnight CT t...,1,0.35,0.65


In [65]:
model_predicts = []
for row in range(len(combined_posts_df)):
    pred_promo = combined_posts_df['pred_promo'][row]
    pred_not_promo = combined_posts_df['pred_not_promo'][row]
    if pred_promo > pred_not_promo:
        model_predicts.append(1)
    else:
        model_predicts.append(0)
        
model_predicts

[1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 0,
 1,
 1,
 1,
 1,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 1,
 1,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0]

In [66]:
combined_posts_df['model_predicts'] = model_predicts
combined_posts_df.head()

,comment,promo,pred_not_promo,pred_promo,model_predicts
0,Summer is a great time to freshen up the kids’...,1,0.01,0.99,1
1,They all look up to you so much @chipgaines Yo...,0,1.00,0.00,0
2,Happy Father's Day to the plant daddy of all p...,0,1.00,0.00,0
3,Y’all really stepped up to the #ChipInChalleng...,0,0.26,0.74,1
4,One more push y’all! We have til midnight CT t...,1,0.35,0.65,1


In [67]:
correct_pred = []
for row in range(len(combined_posts_df)):
    true_class = combined_posts_df['promo'][row]
    model_pred = combined_posts_df['model_predicts'][row]
    if true_class == model_pred:
        correct_pred.append(True)
    else:
        correct_pred.append(False)
        
correct_pred

[True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 False,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 False,
 False,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 True,
 False,
 True,
 False,
 True,
 False,
 

In [69]:
combined_posts_df['correct'] = correct_pred
combined_posts_df.head(10)

,comment,promo,pred_not_promo,pred_promo,model_predicts,correct
0,Summer is a great time to freshen up the kids’...,1,0.01,0.99,1,True
1,They all look up to you so much @chipgaines Yo...,0,1.00,0.00,0,True
2,Happy Father's Day to the plant daddy of all p...,0,1.00,0.00,0,True
3,Y’all really stepped up to the #ChipInChalleng...,0,0.26,0.74,1,False
4,One more push y’all! We have til midnight CT t...,1,0.35,0.65,1,True
5,After months of developing and finalizing reci...,1,0.00,1.00,1,True
6,When @chipgaines and I got the chance to meet ...,1,1.00,0.00,0,False
7,Course complete! School looks good on you @chi...,0,0.07,0.93,1,False
8,What we witnessed this week at @harvardhbs was...,0,0.00,1.00,1,False
9,16 years and it feels like we're just getting ...,0,0.10,0.90,1,False


In [70]:
false_predictions = combined_posts_df.loc[combined_posts_df['correct'] == False]

In [80]:
false_pred_comments = [(comment, promo) for comment, promo in zip(false_predictions['comment'], false_predictions['promo'])]
false_pred_comments

[('Y’all really stepped up to the #ChipInChallenge and together we raised over $2 million for the fight against cancer! We go to @StJude next week and can’t wait to see the precious kids and present them with a check for $1.49 million from all of you. On top of that, $512,000 is going to our dear friend Gabe’s foundation, @bravelikegabe. She will forever inspire us to be brave. ❤️ Thank you ALL for chipping in with us! #NotTodayCancer',
  0),
 ('When @chipgaines and I got the chance to meet some of the kids at @stjude a couple of years ago, we walked away completely changed. These kids were so full of joy and hope—I’ll never forget it. That year, you all helped us raise $230,000 for those kids, and we want to do it again—but go even bigger! We’ve pulled a team together to help us with what we’re calling the #ChipInChallenge, and we hope you’ll be a part of helping us change lives. Head over to the link in profile to find out how you can help.',
  1),
 ('Course complete! School looks go

In [78]:
79/248

0.3185483870967742